In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os,sys

os.getcwd()
rpath = os.path.abspath('../')
if rpath not in sys.path:
    sys.path.insert(0,rpath)

In [3]:
data_path = os.path.join(rpath,'data','data.csv')
data = pd.read_csv(data_path)
data['TransactionStartTime']=pd.to_datetime(data['TransactionStartTime'], format='mixed')
data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0


Create Aggregate Features


In [4]:
# Grouping by AccountId or CustomerId to calculate the total transaction amount
data['TotalTransactionAmount'] = data.groupby('CustomerId')['Amount'].transform('sum')
# Calculating the average transaction amount per customer
data['AverageTransactionAmount'] = data.groupby('CustomerId')['Amount'].transform('mean')
# Counting the number of transactions per customer
data['TransactionCount'] = data.groupby('CustomerId')['TransactionId'].transform('count')
# Calculating the standard deviation of transaction amounts per customer
data['TransactionAmountStdDev'] = data.groupby('CustomerId')['Amount'].transform('std')

In [5]:
customer_aggregates = data.groupby('CustomerId').agg(
    TotalTransactionAmount=('Amount', 'sum'),
    AverageTransactionAmount=('Amount', 'mean'),
    TransactionCount=('TransactionId', 'count'),
    TransactionAmountStdDev=('Amount', 'std'),
    MinTransactionAmount=('Amount', 'min'),
    MaxTransactionAmount=('Amount', 'max'),
    FirstTransactionDate=('TransactionStartTime', 'min'),
    LastTransactionDate=('TransactionStartTime', 'max')
).reset_index()
customer_aggregates

,CustomerId,TotalTransactionAmount,AverageTransactionAmount,TransactionCount,TransactionAmountStdDev,MinTransactionAmount,MaxTransactionAmount,FirstTransactionDate,LastTransactionDate
0,CustomerId_1,-10000.0,-10000.000000,1,NaN,-10000.0,-10000.0,2018-11-21 16:49:14+00:00,2018-11-21 16:49:14+00:00
1,CustomerId_10,-10000.0,-10000.000000,1,NaN,-10000.0,-10000.0,2018-11-21 16:49:09+00:00,2018-11-21 16:49:09+00:00
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333,-5000.0,10000.0,2018-11-16 07:53:19+00:00,2018-11-16 08:20:39+00:00
3,CustomerId_1002,4225.0,384.090909,11,560.498966,-75.0,1500.0,2018-11-15 18:50:09+00:00,2019-01-18 10:05:00+00:00
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146,-5000.0,10000.0,2019-02-01 14:58:07+00:00,2019-02-01 15:04:51+00:00
...,...,...,...,...,...,...,...,...,...
3737,CustomerId_992,20000.0,3333.333333,6,6088.240030,-5000.0,10000.0,2019-02-08 10:23:23+00:00,2019-02-08 10:27:42+00:00
3738,CustomerId_993,20000.0,4000.000000,5,6745.368782,-5000.0,10000.0,2019-01-18 15:51:50+00:00,2019-01-18 15:56:30+00:00
3739,CustomerId_994,543873.0,5384.881188,101,14800.656784,-5000.0,90000.0,2018-11-15 18:37:07+00:00,2019-02-12 11:17:08+00:00
3740,CustomerId_996,139000.0,8176.470588,17,4433.329648,-5000.0,10000.0,2018-12-07 14:37:54+00:00,2018-12-07 15:24:31+00:00
